In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
import keras
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from numpy import array
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import concatenate

In [11]:
train_df = pd.read_csv('LLM_vs_human_train_50k.csv')
val_df = pd.read_csv('LLM_vs_human_val_1k.csv')
# test_df = pd.read_csv('LLM_vs_human_test_1k.csv')

In [13]:
train_df.drop('Unnamed: 0',axis = 1, inplace = True)
train_df.reset_index(inplace = True)
train_df.drop('index',axis = 1, inplace = True)
val_df.drop('Unnamed: 0',axis = 1, inplace = True)
val_df.reset_index(inplace = True)
val_df.drop('index',axis = 1, inplace = True)
# test_df.drop('Unnamed: 0',axis = 1, inplace = True)
# test_df.reset_index(inplace = True)
# test_df.drop('index',axis = 1, inplace = True)

In [14]:
train_df

,text,source,label
0,I'm not happy with what I've wrote to be hones...,Human,0
1,Postmodernist Anthropology and its Impact on T...,Human,0
2,"Firstly, is it okay to write an essay or does ...",Human,0
3,Nissan Company’s Operational Changes and Manag...,Human,0
4,I think some people might find visiting a nati...,Human,0
...,...,...,...
49995,The exploration of space has always captivated...,GPT-3.5,1
49996,"Famous people, whether they are actors, athlet...",GPT-3.5,1
49997,The future of 3D printing and its applications...,GPT-3.5,1
49998,The world is currently facing a critical issue...,GPT-3.5,1


In [15]:
# Human : 0, LLM : 1

In [16]:
txt_train = train_df['text'].astype(str).values
lbl_train = train_df['label'].values
txt_val = val_df['text'].astype(str).values
lbl_val = val_df['label'].values
# txt_test = test_df['text'].astype(str).values
# lbl_test = test_df['label'].values

In [17]:
# train_df['text'].str

In [18]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(txt_train)
token_train = tokenizer.texts_to_sequences(txt_train)
token_val = tokenizer.texts_to_sequences(txt_val)
# token_test = tokenizer.texts_to_sequences(txt_test)
vocab_size = len(tokenizer.word_index) + 1  
print(txt_train[1])
print(token_train[1])

Postmodernist Anthropology and its Impact on Translation Practice Research Paper

It would be fair enough to state that translation in its practical use appeared together with the appearance of the mankind. Not only finding the lexical equivalents, but also interpreting one’s thoughts and ideas, and conveying the meaning of different signs were the tasks of translation at that time. Nowadays the process of translation is even more complex, and simply knowing two languages does not make one a translator.

What is more, with the development of the humanity, more and more translation problems occur, and the number of skills needed to translate is increasing constantly. As it can be seen, translation is realized by humans and for humans, which means that these two notions are inseparable. For this reason, the interdependence between the translation and anthropology, the study of humans, is hard to overestimate.

Indeed, one of the fundamental tasks of anthropology is understanding differen

In [21]:
max_len = 1000

In [22]:
pad_token_train = pad_sequences(token_train, padding='post', maxlen=max_len)
pad_token_val = pad_sequences(token_val, padding='post', maxlen=max_len)
# pad_token_test = pad_sequences(token_test, padding='post', maxlen=max_len)
print(pad_token_train[0, :]) 

[ 669   26 1474   13   82 1644 3744    4   16 4269   22  475  302   32
  162 2036   38 2755  722    2 3176   35   47  826    5    6 1457 1096
   47  930    1   81    1 1096   23 1628   13    6    2   10   45    5
   31 1845    2 1326   62    1  826  154   45  545   47 1149    4  178
   47  912 1447   25  364  614    6  424  874    1  722    2 2688   30
  111    6   63   26    6   68   84   27   61  814    6   68   84    4
  274    4 1366  135    6  226    1 3791  358    4   11   19   68 1437
 3942  109   39 3791   59  699   13    1  125  367  170    6  226  760
   32  158  641   10  179   10   39  974  288   26  699   13    1  125
   27  510  367   45    6  657   63   20   51    6 2835   38   47  722
   12  919  560 2251 1789 2755    1  222   47  416   12    1  826 2484
    3  779    4  838  851 1600 2268    3  649  726   45 3864   47 1375
  622   24    1  826 2484  267    1 1139   30 2505    5    1 2966    2
    1 1096 2201  121  250 3852    4    1 1518   85  236   80   23   26
   46 

In [25]:
len(pad_token_train)

50000

In [26]:
vocab_size

202332

In [27]:
def define_model(length, vocab_size):
 
 inputs = Input(shape=(length,))
 embedding1 = Embedding(vocab_size, 1024)(inputs)
 conv1 = Conv1D(filters=2048, kernel_size=4, activation='relu')(embedding1)
 conv2 = Conv1D(filters=1024, kernel_size=4, activation='relu')(conv1)
 conv3 = Conv1D(filters=512, kernel_size=4, activation='relu')(conv2)
 pool1 = MaxPooling1D(pool_size=2)(conv3)
 conv4 = Conv1D(filters=128, kernel_size=4, activation='relu')(pool1)
 conv5 = Conv1D(filters=64, kernel_size=4, activation='relu')(conv4)
 drop1 = Dropout(0.3)(conv5)
 pool2 = MaxPooling1D(pool_size=2)(drop1)
 flat1 = Flatten()(pool2)
 dense1 = Dense(100, activation='relu')(flat1)
 outputs = Dense(1, activation='sigmoid')(dense1)
 model = Model(inputs= inputs, outputs=outputs)
 # compile
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

 return model


In [28]:
model = define_model(max_len, vocab_size)

In [29]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 1000, 1024)     │   207,187,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 997, 2048)      │     8,390,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 994, 1024)      │     8,389,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 991, 512)       │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 495, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 492, 128)       │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 489, 64)        │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 489, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 244, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15616)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     1,561,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 227,922,825 (869.46 MB)

 Trainable params: 227,922,825 (869.46 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.fit(pad_token_train, lbl_train,validation_data=(pad_token_val, lbl_val), epochs=10, batch_size=32, verbose=1)

Epoch 1/10


c:\Users\gehna\Documents\MSCS\Sem2\Deep_Learning\AI_text_detector\ai_det\Lib\site-packages\tensorflow\python\framework\indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 207187968 elements. This may consume a large amount of memory.
  warnings.warn(


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18833s 12s/step - accuracy: 0.8592 - loss: 0.3245 - val_accuracy: 0.9550 - val_loss: 0.1141
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21446s 14s/step - accuracy: 0.9657 - loss: 0.0928 - val_accuracy: 0.9540 - val_loss: 0.1055
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18553s 12s/step - accuracy: 0.9751 - loss: 0.0666 - val_accuracy: 0.9610 - val_loss: 0.1106
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18557s 12s/step - accuracy: 0.9781 - loss: 0.0622 - val_accuracy: 0.9640 - val_loss: 0.0989
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18632s 12s/step - accuracy: 0.9857 - loss: 0.0406 - val_accuracy: 0.9570 - val_loss: 0.1089
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16627s 11s/step - accuracy: 0.9876 - loss: 0.0402 - val_accuracy: 0.9640 - val_loss: 0.1005
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16143s 10s/step - accuracy: 0.9924 - loss: 0.0237 - val_accuracy: 0.9570 - val_loss: 0.1297
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15858s 10s/step - accuracy: 0.9

In [31]:
model.save('cnn_50k.keras', overwrite= True)

In [32]:
# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)